# 🎮 Fantasy Football Database - Master Control Panel

This notebook provides complete control over your fantasy football projections database.

**Features:**
- 🗑️ Database management (view, clear, reset)
- 📊 Run scrapers individually or all at once
- 📈 View data quality metrics
- 🔍 Check team coverage by source


## 📦 Setup & Imports

**Important Notes:**
- The FanDuel scraper (Playwright-based) runs via subprocess to avoid conflicts with Jupyter's event loop. All other scrapers run directly in the notebook.
- **⚠️ If you modify any scraper files (scraper_*.py), you MUST restart the Jupyter kernel to reload the changes!** Python caches imported modules.


In [ ]:
import os
import sqlite3
import subprocess
import sys
from datetime import datetime
from pathlib import Path

# Add parent directory to path for imports
sys.path.append(str(Path().absolute().parent.parent))
sys.path.append(str(Path().absolute().parent / 'scrapers'))

from backend.scrapers.database import ProjectionsDB
from backend.scrapers.scraper_sleeper import SleeperScraper
from backend.scrapers.scraper_espn import ESPNScraper
from backend.scrapers.scraper_fantasypros import FantasyProsScraper
from backend.scrapers.scraper_firstdown import FirstDownStudioScraper

# Configuration
WEEK = "Week 10"
SEASON = "2025"
HEADLESS = True  # Set to False to see browser

print(f"✓ Imports successful")
print(f"Configuration: {WEEK}, Season {SEASON}")
print(f"Headless mode: {HEADLESS}")
print(f"\n⚠️  Note: FanDuel scraper must be run via subprocess (Playwright limitation in Jupyter)")


✓ Imports successful
Configuration: Week 10, Season 2025
Headless mode: True

⚠️  Note: FanDuel scraper must be run via subprocess (Playwright limitation in Jupyter)


## 📊 Database Status & Overview


## 📊 Week × Source Matrix View

Quick diagnostic to see which sources have which weeks.


In [ ]:
def show_week_source_matrix():
    """Display a matrix showing record counts by week and source."""
    import pandas as pd
    
    conn = sqlite3.connect("../data/databases/projections.db")
    
    query = """
        SELECT 
            week,
            source_website,
            COUNT(*) as count
        FROM projections
        GROUP BY week, source_website
        ORDER BY week, source_website
    """
    
    df = pd.read_sql_query(query, conn)
    conn.close()
    
    if len(df) == 0:
        print("No data in database")
        return
    
    # Create pivot table
    pivot = df.pivot(index='source_website', columns='week', values='count')
    pivot = pivot.fillna(0).astype(int)
    
    print("\n" + "="*70)
    print("WEEK × SOURCE MATRIX")
    print("="*70)
    print(pivot.to_string())
    print("="*70 + "\n")

show_week_source_matrix()



WEEK × SOURCE MATRIX
week             Week 10
source_website          
espn.com             235
fanduel.com          472
fantasypros.com      177
sleeper.com          429



In [3]:
def show_database_status():
    """Display current database status."""
    with ProjectionsDB() as db:
        all_projs = db.get_projections()
        
        print(f"\n{'='*70}")
        print(f"DATABASE STATUS")
        print(f"{'='*70}\n")
        print(f"Total records: {len(all_projs)}")
        
        # Count by source
        print(f"\n📊 Records by Source:")
        print("-" * 70)
        sources = {}
        for p in all_projs:
            source = p['source_website']
            sources[source] = sources.get(source, 0) + 1
        
        for source, count in sorted(sources.items()):
            print(f"  {source:<25} {count:>5} records")
        
        # Count by week
        print(f"\n📅 Records by Week:")
        print("-" * 70)
        weeks = {}
        for p in all_projs:
            week = p['week']
            weeks[week] = weeks.get(week, 0) + 1
        
        for week, count in sorted(weeks.items()):
            print(f"  {week:<25} {count:>5} records")
        
        # Team coverage
        print(f"\n🏈 Team Data Coverage:")
        print("-" * 70)
        for source in sorted(sources.keys()):
            source_projs = [p for p in all_projs if p['source_website'] == source]
            with_team = sum(1 for p in source_projs if p.get('team'))
            total = len(source_projs)
            pct = (with_team / total * 100) if total > 0 else 0
            status = "✅" if pct >= 95 else "⚠️" if pct >= 75 else "❌"
            print(f"  {source:<25} {with_team:>4}/{total:<4} ({pct:>5.1f}%) {status}")

show_database_status()



DATABASE STATUS

Total records: 0

📊 Records by Source:
----------------------------------------------------------------------

📅 Records by Week:
----------------------------------------------------------------------

🏈 Team Data Coverage:
----------------------------------------------------------------------


## 🗑️ Database Management

### ⚠️ Clear Entire Database
**WARNING:** This will delete ALL records from the database!


In [ ]:
def clear_entire_database():
    """Delete all records from the database."""
    confirm = input("⚠️  Are you sure you want to delete ALL records? Type 'DELETE ALL' to confirm: ")
    
    if confirm == "DELETE ALL":
        conn = sqlite3.connect("../data/databases/projections.db")
        cursor = conn.cursor()
        cursor.execute("DELETE FROM projections")
        conn.commit()
        deleted = cursor.rowcount
        conn.close()
        print(f"\n✓ Deleted {deleted} records from database")
        print("Database is now empty.")
    else:
        print("\n✗ Cancelled. Database unchanged.")

# Uncomment to run:
clear_entire_database()
show_database_status()


### 🗑️ Clear Specific Source


In [ ]:
def clear_source(source_name, week=None):
    """Delete records from a specific source."""
    conn = sqlite3.connect("../data/databases/projections.db")
    cursor = conn.cursor()
    
    if week:
        cursor.execute("DELETE FROM projections WHERE source_website = ? AND week = ?", (source_name, week))
        print(f"✓ Deleted {cursor.rowcount} records from {source_name} for {week}")
    else:
        cursor.execute("DELETE FROM projections WHERE source_website = ?", (source_name,))
        print(f"✓ Deleted {cursor.rowcount} records from {source_name} (all weeks)")
    
    conn.commit()
    conn.close()

# Examples:
# clear_source("fantasypros.com", WEEK)
# clear_source("fantasypros.com")
# show_database_status()


### 🗑️ Clear Specific Week

Delete all records from a specific week across all sources.


In [ ]:
def clear_week(week_name):
    """Delete all records from a specific week across all sources."""
    conn = sqlite3.connect("../data/databases/projections.db")
    cursor = conn.cursor()
    
    cursor.execute("DELETE FROM projections WHERE week = ?", (week_name,))
    deleted = cursor.rowcount
    conn.commit()
    conn.close()
    
    print(f"✓ Deleted {deleted} records from {week_name}")

# Example: clear_week("Week 8")
# Uncomment to run:
clear_week("Week 8")
show_database_status()

## 🤖 Run Scrapers Individually

**Note:** FanDuel uses Playwright which conflicts with Jupyter's event loop, so it runs via subprocess (separate process). The other scrapers run directly in the notebook.

### 1️⃣ FanDuel Scraper


In [4]:
print(f"\n{'='*70}")
print(f"🔵 FANDUEL SCRAPER (via subprocess)")
print(f"{'='*70}\n")
print(f"Using configuration: {WEEK}, Headless={HEADLESS}\n")

# FanDuel uses Playwright which doesn't work in Jupyter's event loop
# Run it as a subprocess instead
try:
    # Create a temporary script to run FanDuel scraper
    # Use proper string formatting to pass variables
    script_content = f'''from scraper_fanduel import FanDuelScraper

print("Starting FanDuel scraper...")
print(f"Week: {WEEK}")
print(f"Headless: {HEADLESS}")

with FanDuelScraper(headless={HEADLESS}) as scraper:
    scraper.scrape_and_save(week="{WEEK}")
'''
    
    # Write to temp file
    with open("_temp_fanduel.py", "w") as f:
        f.write(script_content)
    
    # Debug: Show what we're running
    print(f"Running FanDuel scraper for {WEEK}...\n")
    
    # Run as subprocess
    result = subprocess.run(
        [sys.executable, "_temp_fanduel.py"],
        capture_output=True,
        text=True
    )
    
    # Print output
    print(result.stdout)
    
    if result.returncode == 0:
        print("\n✅ FanDuel scraping complete!")
    else:
        print(f"\n❌ Error: {result.stderr}")
    
    # Clean up temp file
    if os.path.exists("_temp_fanduel.py"):
        os.remove("_temp_fanduel.py")
        
except Exception as e:
    print(f"\n❌ Error: {e}")
    # Clean up temp file if error
    if os.path.exists("_temp_fanduel.py"):
        os.remove("_temp_fanduel.py")

show_database_status()



🔵 FANDUEL SCRAPER (via subprocess)

Using configuration: Week 10, Headless=True

Running FanDuel scraper for Week 10...

Starting FanDuel scraper...
Week: Week 10
Headless: True
Navigating to https://www.fanduel.com/research/nfl/fantasy/ppr...
  Intercepted data from https://fdresearch-api.fanduel.com/graphql
  âœ“ Found 472 projections in response

Parsing 472 projections...
  Josh Allen (QB): 23.2 pts
  Lamar Jackson (QB): 23.0 pts
  Christian McCaffrey (RB): 22.6 pts
  Bijan Robinson (RB): 22.1 pts
  Jonathan Taylor (RB): 21.6 pts
  Puka Nacua (WR): 21.4 pts
  De'Von Achane (RB): 21.3 pts
  James Cook (RB): 19.8 pts
  Jahmyr Gibbs (RB): 19.7 pts
  Drake Maye (QB): 19.4 pts
  Caleb Williams (QB): 19.1 pts
  Justin Herbert (QB): 19.1 pts
  Brock Purdy (QB): 19.0 pts
  Jalen Hurts (QB): 18.9 pts
  Matthew Stafford (QB): 18.9 pts
  Bo Nix (QB): 18.8 pts
  Baker Mayfield (QB): 18.8 pts
  Amon-Ra St. Brown (WR): 18.8 pts
  Jaxson Dart (QB): 18.7 pts
  Daniel Jones (QB): 18.5 pts
  Jared 

### 2️⃣ Sleeper Scraper


In [5]:
print(f"\n{'='*70}")
print(f"💤 SLEEPER SCRAPER")
print(f"{'='*70}\n")

try:
    with SleeperScraper() as scraper:
        scraper.scrape_and_save(week=WEEK, season=SEASON)
    print("\n✅ Sleeper scraping complete!")
except Exception as e:
    print(f"\n❌ Error: {e}")

show_database_status()



💤 SLEEPER SCRAPER


Fetching Sleeper projections for 2025 Week 10...
Fetching all players from Sleeper API...
  ✓ Loaded 11400 players
Fetching from: https://api.sleeper.app/v1/projections/nfl/regular/2025/10
  ✓ Successfully fetched 8618 player projections

Parsing 8618 projections...
  Bucky Irving (RB): 16.0 pts
  Oronde Gadsden (TE): 12.1 pts
  Matthew Stafford (QB): 19.6 pts
  Khalil Shakir (WR): 12.3 pts
  James Cook (RB): 16.9 pts
  Jauan Jennings (WR): 11.1 pts
  Sam Darnold (QB): 19.4 pts
  Kimani Vidal (RB): 12.7 pts
  Jaxon Smith-Njigba (WR): 23.1 pts
  Jaylen Waddle (WR): 14.8 pts
  Daniel Jones (QB): 21.4 pts
  D'Andre Swift (RB): 12.9 pts
  Geno Smith (QB): 17.8 pts
  Tua Tagovailoa (QB): 18.5 pts
  Jalen Hurts (QB): 22.6 pts
  Ladd McConkey (WR): 13.7 pts
  Rome Odunze (WR): 12.3 pts
  Jordan Love (QB): 18.4 pts
  Justin Herbert (QB): 20.9 pts
  Sam LaPorta (TE): 13.0 pts
  Lamar Jackson (QB): 25.1 pts
  Davante Adams (WR): 15.1 pts
  Aaron Rodgers (QB): 18.0 pts
  Deeb

### 3️⃣ ESPN Scraper


In [6]:
print(f"\n{'='*70}")
print(f"🔴 ESPN SCRAPER")
print(f"{'='*70}\n")

try:
    with ESPNScraper(headless=HEADLESS) as scraper:
        scraper.scrape_and_save(week=WEEK, season=SEASON)
    print("\n✅ ESPN scraping complete!")
except Exception as e:
    print(f"\n❌ Error: {e}")

show_database_status()



🔴 ESPN SCRAPER


Fetching ESPN projections for Week 10...
Navigating to https://fantasy.espn.com/football/players/projections...
Clicking 'Sortable Projections' view...
  ✓ Switched to Sortable Projections view
Setting scoring to Points PPR...
  ✓ Set to Points PPR
Setting to This Week projections...
  ✓ Set to This Week

Scraping 6 positions (up to 50 players each)...

  Filtering to QB...
    ✓ Filtered to QB
    Found 50 players
      Lamar Jackson (BAL): 23.6 pts
      Jaxson Dart (NYG): 23.4 pts
      Josh Allen (BUF): 22.5 pts
      Jalen Hurts (PHI): 21.4 pts
      Drake Maye (NE): 20.6 pts
      Justin Herbert (LAC): 20.1 pts
      Daniel Jones (IND): 20.0 pts
      Matthew Stafford (LAR): 19.6 pts
      Caleb Williams (CHI): 19.5 pts
      Bo Nix (DEN): 19.2 pts
      Jared Goff (DET): 18.7 pts
      J.J. McCarthy (MIN): 17.7 pts
      Brock Purdy (SF): 17.1 pts
      Jordan Love (GB): 17.0 pts
      Justin Fields (NYJ): 16.7 pts
      Sam Darnold (SEA): 16.6 pts
      Marcus

### 4️⃣ FantasyPros Scraper


In [7]:
print(f"\n{'='*70}")
print(f"📊 FANTASYPROS SCRAPER")
print(f"{'='*70}\n")

try:
    with FantasyProsScraper(headless=HEADLESS) as scraper:
        scraper.scrape_and_save(week=WEEK)
    print("\n✅ FantasyPros scraping complete!")
except Exception as e:
    print(f"\n❌ Error: {e}")

show_database_status()



📊 FANTASYPROS SCRAPER


Fetching FantasyPros consensus rankings for Week 10...

Scraping QB from https://www.fantasypros.com/nfl/rankings/qb.php...
  Found 55 rows
  Josh Allen (BUF): 25.0 pts
  Lamar Jackson (BAL): 24.5 pts
  Drake Maye (NE): 24.0 pts
  Justin Herbert (LAC): 23.5 pts
  Jalen Hurts (PHI): 23.0 pts
  Bo Nix (DEN): 22.5 pts
  Jaxson Dart (NYG): 22.0 pts
  Baker Mayfield (TB): 21.5 pts
  Caleb Williams (CHI): 21.0 pts
  Daniel Jones (IND): 20.5 pts
  Jared Goff (DET): 20.0 pts
  Matthew Stafford (LAR): 19.5 pts
  Sam Darnold (SEA): 19.0 pts
  Jordan Love (GB): 18.5 pts
  J.J. McCarthy (MIN): 18.0 pts
  Justin Fields (NYJ): 17.5 pts
  Marcus Mariota (WAS): 17.0 pts
  Aaron Rodgers (PIT): 16.5 pts
  Jacoby Brissett (ARI): 16.0 pts
  Bryce Young (CAR): 15.5 pts
  Michael Penix Jr. (ATL): 15.0 pts
  Tua Tagovailoa (MIA): 14.5 pts
  Mac Jones (SF): 14.0 pts
  Trevor Lawrence (JAC): 13.5 pts
  Geno Smith (LV): 13.0 pts
  Tyler Shough (NO): 12.5 pts
  Dillon Gabriel (CLE): 12.0

### 5️⃣ First Down Studio Scraper


In [ ]:
print(f"\n{'='*70}")
print(f"🎯 FIRST DOWN STUDIO SCRAPER")
print(f"{'='*70}\n")

try:
    with FirstDownStudioScraper(headless=HEADLESS) as scraper:
        scraper.scrape_and_save(week=WEEK, scoring="PPR")
    print("\n✅ First Down Studio scraping complete!")
except Exception as e:
    print(f"\n❌ Error: {e}")

show_database_status()



🎯 FIRST DOWN STUDIO SCRAPER

Navigating to https://www.firstdown.studio/rankings...
Note: PPR points will be calculated from component stats for FLEX players

Scraping QB tab...
Error scraping QB tab: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff6874ae8e5+80021]
	GetHandleVerifier [0x0x7ff6874ae940+80112]
	(No symbol) [0x0x7ff68723060f]
	(No symbol) [0x0x7ff687288854]
	(No symbol) [0x0x7ff687288b1c]
	(No symbol) [0x0x7ff6872dc927]
	(No symbol) [0x0x7ff6872b126f]
	(No symbol) [0x0x7ff6872d968a]
	(No symbol) [0x0x7ff6872b1003]
	(No symbol) [0x0x7ff6872795d1]
	(No symbol) [0x0x7ff68727a3f3]
	GetHandleVerifier [0x0x7ff68776dc7d+2960429]
	GetHandleVerifier [0x0x7ff687767f3a+2936554]
	GetHandleVerifier [0x0x7ff687788977+3070247]
	GetHandleVerifier [0x0x7ff6874c83ce+185214]
	GetHandleVerifier [0x0x7ff6874cfe1f+216527]
	GetHandleVerifier [0x0x7ff6874b7b24+117460]
	GetHandleVerifier [0x0x7ff6874b7cdf+117903]
	GetHandleVerifier [0x0x7ff68749dbb8+11112]
	BaseThreadInitThunk [0x0x7ffc16b2e8d7

## 🎯 Quick Reference

### Useful Commands

```python
# View current database status
show_database_status()

# Clear entire database (requires confirmation)
clear_entire_database()

# Clear specific source
clear_source("fantasypros.com", WEEK)  # Clear specific week
clear_source("fanduel.com")            # Clear all weeks

# Change configuration
WEEK = "Week 9"
SEASON = "2024"
HEADLESS = False  # Show browser
```

### Source Names
- `"fanduel.com"`
- `"sleeper.com"`
- `"espn.com"`
- `"fantasypros.com"`
- `"firstdown.studio"`
